# ニューラルネットワークの動作確認（part 4）

<a href="http://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier"><b>sklearn.neural_network.MLPClassifier</b></a> を使用します。

グリッドサーチにより、条件を変えてみて動きを確認してみます。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_daikin_conversation.csv',
    'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_daikin_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]
CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_ptna_conversation.csv]


## (2) グリッドサーチ

学習(GridSearchCV.fit)--->評価(Evaluator.evaluate)の流れで実行させ、実行時間計測、およびaccuracy計測を行います。

In [3]:
from time import time
import numpy as np

from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier

from learning.core.evaluator import Evaluator

def fit_and_cross_validation(path):
    '''
        訓練データのTF-IDFベクターを作成
    '''
    basename = os.path.basename(path)
    print("prepare_tf_idf_vectors: dataset=%s..." % basename)
    t0 = time()

    X, y, vectorizer = TestTool.prepare_tf_idf_vectors(path)
    print("prepare_tf_idf_vectors: done in %0.3fs." % (time() - t0))

    '''
        訓練データ全体を使用して学習実施
        プロダクションと同様、ハイパーパラメータの選択のために
        グリッドサーチを使用します
        
        ハイパーパラメータとして、レイヤー／ユニット数を採用
            レイヤー数：デフォルトの1層から、4層まで変化させます
            ユニット数：デフォルト100／その半分50／その倍200の３通り
    '''
    print("GridSearchCV: search and fitting...")
    t0 = time()

    params = {
        'hidden_layer_sizes': [
            (50,),(100,),(200,), # 1層
            (50,50,),(100,100,),(200,200,), # 2層
            (50,50,50,),(100,100,100,),(200,200,200,), # 3層
            (50,50,50,50,),(100,100,100,100,),(200,200,200,200,), # 4層
        ]
    }

    grid = GridSearchCV(
        MLPClassifier(activation='logistic', max_iter=10000, shuffle=False, random_state=0), 
        param_grid=params
    )
    grid.fit(X, y)
    estimator = grid.best_estimator_
    print("GridSearchCV: done in %0.3fs." % (time() - t0))

    ''' 
        クロスバリデーション（モデル評価フェーズ）を実施
        プロダクションと同様、Evaluator クラスを使用して評価します
        
        Evaluator クラスで使用している cross_val_score 関数は、
        引数の estimator により、
        内部で fit, predict, predict_proba の各関数を実行しています。
    '''
    print("Evaluator: evaluating...")
    t0 = time()

    evaluator = Evaluator()
    evaluator.evaluate(estimator, X, y, threshold=0.5)
    print("Evaluator: done in %0.3fs." % (time() - t0))
    
    return (basename, X, y, vectorizer, estimator, evaluator)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### (2-1) グリッドサーチ実行

所要時間を計測しました。

- test_daikin_conversation.csv：data=17,443件、GridSearchCV=9042s、Evaluator＝145s

- test_benefitone_conversation.csv：data=4,114件、GridSearchCV=691s、Evaluator＝18s

- test_septeni_conversation.csv：data=2,156件、GridSearchCV=441s、Evaluator＝13s

- test_ptna_conversation.csv：data=4,559件、GridSearchCV=414s、Evaluator＝13s


In [4]:
list_of_classifiers = []
for path in temp_path:
    classifier = fit_and_cross_validation(path)
    list_of_classifiers.append(classifier)

2017/04/05 PM 01:26:02 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017/04/05 PM 01:26:02 TextArray#__init__ start


prepare_tf_idf_vectors: dataset=test_daikin_conversation.csv...


2017/04/05 PM 01:26:12 TextArray#to_vec start
2017/04/05 PM 01:26:13 TextArray#to_vec end
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


prepare_tf_idf_vectors: done in 10.915s.
GridSearchCV: search and fitting...


2017/04/05 PM 03:56:54 self.threshold: 0.5


...

prepare_tf_idf_vectors: done in 1.820s.
GridSearchCV: search and fitting...


2017/04/05 PM 04:25:41 self.threshold: 0.5


GridSearchCV: done in 414.226s.
Evaluator: evaluating...


2017/04/05 PM 04:25:54 Evaluator#evaluate#elapsed time: 13486.329794 ms
2017/04/05 PM 04:25:54 accuracy: 0.978965819457


0.978965819457
Evaluator: done in 13.489s.


### (2-2) 選択されたモデルを確認

In [5]:
for classifier in list_of_classifiers:
    basename, X, y, vectorizer, estimator, evaluator = classifier
    params = estimator.get_params()

    print('[%s] best parameter: hidden_layer_sizes=%s, accuracy=%0.6f' % (
        basename, str(params['hidden_layer_sizes']), evaluator.accuracy
    ))

[test_daikin_conversation.csv] best parameter: hidden_layer_sizes=(100,), accuracy=0.985557
[test_benefitone_conversation.csv] best parameter: hidden_layer_sizes=(50, 50), accuracy=0.985437
[test_septeni_conversation.csv] best parameter: hidden_layer_sizes=(100,), accuracy=0.974074
[test_ptna_conversation.csv] best parameter: hidden_layer_sizes=(50, 50), accuracy=0.978966
